### 以港铁路线为例探究广度优先搜索
#### 1.爬取百度百科港铁线路——共11条，10条可拓展到站点信息，机场快线无站点信息，因此从机场官网爬取
#### 2.数据清洗：对爬取的线路信息进行处理，仅保留站点信息
#### 3.建立搜索图：获取站点关联信息
#### 4.breath first search：输入起始站和终点站，输出换乘路线

In [1]:
import requests
import re
import pandas as pd

### 1.1 爬取百科港铁页面11个站点及url

In [2]:
#请求头
header_baike = {
'Accept': \
'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'zh-CN,zh;q=0.9',
'Cache-Control': 'max-age=0',
'Connection': 'keep-alive',
'Cookie': 'BAIDUID=7B376B31D697A0E75D27E59240AA6122:FG=1; BIDUPSID=7B376B31D697A0E75D27E59240AA6122; PSTM=1532439485; BDUSS=I1N29scG5OQ1hNa3R0dURONnE0bTU0YTZrVXVjU0J5MUtmWHNFZn43V2Q0ZXhiQVFBQUFBJCQAAAAAAAAAAAEAAAB61GYS0PnO6MC7AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAJ1UxVudVMVbOU; BDORZ=B490B5EBF6F3CD402E515D22BCDA1598; delPer=0; PSINO=1; ZD_ENTRY=baidu; pgv_pvi=8564731904; pgv_si=s7417086976; H_PS_PSSID=1464_21094_29522_29521_28519_29098_28834_29220; BDRCVFR[feWj1Vr5u3D]=I67x6TjHwwYf0; Hm_lvt_55b574651fcae74b0a9f1cf9c8d7c93a=1563115523,1563166624,1563207180,1563296120; Hm_lpvt_55b574651fcae74b0a9f1cf9c8d7c93a=1563296120',
'Host': 'baike.baidu.com',
'Referer': 'https://www.baidu.com/link?url=w0s0w-dncOzFCQ_f3IizUUp7WbS1IYZR13_8-Ki752fB0bvCfZZlWm88739VMtBwL9YeW5vYadCorV-uOIaiqqrXSOQdBKx1jxQM_kLxpUW&wd=&eqid=e87456c50000f718000000035d2e0173',
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
}

In [3]:
def get_response(url,header):
    res = requests.get(url,headers=header,allow_redirects=False)
    return res,res.content.decode('utf8')

hk_baike_url = 'https://baike.baidu.com/item/%E6%B8%AF%E9%93%81/7869166?fr=aladdin'  #百科港铁
res_status,content = get_response(hk_baike_url,header_baike)

In [4]:
pattern_lines = re.compile(r'<a target=_blank href="(/item/.*?%BA%BF/\d+)" data-lemmaid="\d+">(\w+)')
raw_lines = re.findall(pattern_lines,content)
metro_lines = {}
for line in raw_lines:
    metro_lines[line[1]] = 'https://baike.baidu.com' + line[0]
    
metro_lines

{'东铁线': 'https://baike.baidu.com/item/%E4%B8%9C%E9%93%81%E7%BA%BF/9405990',
 '观塘线': 'https://baike.baidu.com/item/%E8%A7%82%E5%A1%98%E7%BA%BF/9399850',
 '荃湾线': 'https://baike.baidu.com/item/%E8%8D%83%E6%B9%BE%E7%BA%BF/9403835',
 '港岛线': 'https://baike.baidu.com/item/%E6%B8%AF%E5%B2%9B%E7%BA%BF/9404355',
 '东涌线': 'https://baike.baidu.com/item/%E4%B8%9C%E6%B6%8C%E7%BA%BF/9405092',
 '将军澳线': 'https://baike.baidu.com/item/%E5%B0%86%E5%86%9B%E6%BE%B3%E7%BA%BF/9352103',
 '西铁线': 'https://baike.baidu.com/item/%E8%A5%BF%E9%93%81%E7%BA%BF/9406685',
 '马鞍山线': 'https://baike.baidu.com/item/%E9%A9%AC%E9%9E%8D%E5%B1%B1%E7%BA%BF/9414345',
 '迪士尼线': 'https://baike.baidu.com/item/%E8%BF%AA%E5%A3%AB%E5%B0%BC%E7%BA%BF/5500566',
 '南港岛线': 'https://baike.baidu.com/item/%E5%8D%97%E6%B8%AF%E5%B2%9B%E7%BA%BF/10678930',
 '机场快线': 'https://baike.baidu.com/item/%E6%9C%BA%E5%9C%BA%E5%BF%AB%E7%BA%BF/10349635'}

### 1.2 拓展11条线路的url，爬取站点信息

In [5]:
def get_stations(colname):
    stations = {}
    for name,url in metro_lines.items():
        tables = pd.read_html(url,header = 0)
        for table in tables:
            try:
                stations[name] = table[colname].tolist()
            except:
                continue
    return stations

all_line_stations = {}
for col_name in ['车站名称','站名','站点名称','车站套色同埋名','方向']:
    all_line_stations.update(get_stations(col_name))

all_line_stations.keys()  #未爬取到 机场快线 的站点信息

dict_keys(['东铁线', '观塘线', '港岛线', '东涌线', '西铁线', '荃湾线', '南港岛线', '将军澳线', '马鞍山线', '迪士尼线'])

In [6]:
#从香港国际机场官网地面交通网页爬取机场快线站点信息
to_do = list(metro_lines.keys()-all_line_stations.keys())
'''机场快线'''
ap_url = 'https://www.hongkongairport.com/sc/transport/to-from-airport/airport-express.page'
ap_header = {#请求头
'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'zh-CN,zh;q=0.9',
'cache-control': 'max-age=0',
'cookie': '__cfduid=d69a1e9d82f99d9d2d1c150201667d62f1563251320; HKIA.font-size=S; HKIA.important-notice-status=open; _ga=GA1.2.1641509565.1563251323; JSESSIONID=04F98454996BDEE8B6DCF218484FD456; _gid=GA1.2.1308698360.1563341408; HKIA.weather=%7B%22weather%22%3A%7B%22code%22%3A%22SP%22%2C%22description%22%3A%5B%22Sunny%20Periods%22%2C%22%E9%96%93%E6%9C%89%E9%99%BD%E5%85%89%22%2C%22%E9%97%B4%E6%9C%89%E9%98%B3%E5%85%89%22%5D%2C%22icon%22%3A%22pic51.png%22%7D%2C%22temperature%22%3A33%2C%22humidity%22%3A76%2C%22latestUpdate%22%3A%222019-07-17T05%3A30%3A13.385Z%22%2C%22warningInfo%22%3A%5B%5D%2C%22hasWarning%22%3Afalse%7D',
'upgrade-insecure-requests': '1',
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}

ap_status,ap_res = get_response(ap_url,ap_header)
ap_table = pd.read_html(ap_res,header=0)
all_line_stations[to_do[0]] = ap_table[0].iloc[:,0].tolist()

all_line_stations

{'东铁线': ['罗湖站',
  '落马洲站',
  '上水站',
  '粉岭站',
  '太和站',
  '大埔墟站',
  '大学站',
  '马场站',
  '火炭站',
  '沙田站',
  '大围站',
  '九龙塘站',
  '旺角东站',
  '红磡站',
  '*罗湖站，落马洲站位于边境禁区，仅限持有效旅游证件或边境禁区通行证乘客使用，违者可能会被检控。 *马场站只在沙田或快活谷马场举行赛事或特别节目时开放，18岁以下人士在赛马日请勿在此站出闸。 *大围站临时车站于1983年8月15日启用，现有车站于1986年4月23日启用。 *旺角东站前身为油麻地火车站，于1910年10月1日启用，现有车站于1982年5月4日启用。2007年12月2日改名为旺角东站。'],
 '观塘线': ['调景岭',
  '油塘',
  '蓝田',
  '观塘',
  '牛头角',
  '九龙湾',
  '彩虹',
  '钻石山',
  '黄大仙',
  '乐富',
  '九龙塘',
  '石硖尾',
  '太子（附注3）',
  '旺角（附注3）',
  '油麻地（附注2）',
  '何文田（附注5）',
  '黄埔（附注5）'],
 '港岛线': ['坚尼地城站',
  '香港大学站',
  '西营盘站',
  '上环站',
  '中环站',
  '金钟站',
  '湾仔站',
  '铜锣湾站',
  '天后站',
  '炮台山站',
  '北角站',
  '鲗鱼涌站',
  '太古站',
  '西湾河站',
  '筲箕湾站',
  '杏花邨站',
  '柴湾站',
  '附注 港岛线将会伸延，为西区提供铁路服务，其中坚尼地城站及香港大学站率先于2014年12月28日通车，西营盘站则已于2015年3月启用。'],
 '东涌线': ['添马',
  '会展站',
  '铜锣湾北站',
  '炮台山站（附注3）',
  '北角站（附注3）',
  '鲗鱼涌站（附注3）',
  '太古站（附注3）',
  '西湾河站（附注3）',
  '筲箕湾站（附注3）',
  '杏花邨站（附注3）',
  '柴湾站（附注3）'],
 '西铁线': ['屯门站 Tuen Mun',
  '兆康站 Siu Hong',
  '天水围站 Tin Shui Wai',
  '朗屏站 Long P

### 2.数据清洗


观察各线路站点数据可知，  
1.东铁线、港铁线、西铁线、荃湾线最后一条数据均为附注信息；  
2.观塘线、东涌线部分站点有“（附注）”标识；  
3.西铁线、荃湾线、南港岛线站名为中英文拼接；  
4.不同线路站点名字不统一，比如北角，北角站

In [7]:
'''
0 '东铁线',pop
1 '观塘线',del
2 '港岛线',pop
3 '东涌线',del
4 '西铁线',pop
5 '荃湾线',pop
6 '南港岛线',——4,5,6去英语
7 '将军澳线',
8 '马鞍山线',
9 '迪士尼线',
10 '机场快线'
'''
lines = list(all_line_stations.keys())

def pop_last(dict_all_line_stations,key):#删除最后一条附注信息  #key=lines[0]
    dict_all_line_stations[key].pop(-1)
    print(dict_all_line_stations[key])
    return key,dict_all_line_stations[key]


def del_brackets_eng(dict_all_line_stations,key,s_split='（',is_pattern=True): #删除（附注）和英文
    values=[]
    
    if is_pattern:
        for s in dict_all_line_stations[key]:
            pattern=re.compile(r'[\u4e00-\u9fa5]')  
            value = ''.join(re.findall(pattern,s))
            values.append(value)
    else:
        for s in dict_all_line_stations[key]:
            value = s.split(s_split)[0]
            values.append(value)
    dict_all_line_stations[key]=values
    print(dict_all_line_stations[key])
    return key,dict_all_line_stations[key]

In [8]:
for i in [0,2,4,5]:
    pop_last(all_line_stations,lines[i])

for i in [1,3]:
    del_brackets_eng(all_line_stations,lines[i],'（',is_pattern=False)
 
for i in [4,5,6]:
    del_brackets_eng(all_line_stations,lines[i],is_pattern=True)
    
all_line_stations['马鞍山线'].pop(0)

['罗湖站', '落马洲站', '上水站', '粉岭站', '太和站', '大埔墟站', '大学站', '马场站', '火炭站', '沙田站', '大围站', '九龙塘站', '旺角东站', '红磡站']
['坚尼地城站', '香港大学站', '西营盘站', '上环站', '中环站', '金钟站', '湾仔站', '铜锣湾站', '天后站', '炮台山站', '北角站', '鲗鱼涌站', '太古站', '西湾河站', '筲箕湾站', '杏花邨站', '柴湾站']
['屯门站 Tuen Mun', '兆康站 Siu Hong', '天水围站 Tin Shui Wai', '朗屏站 Long Ping', '元朗站 Yuan Long', '锦上路站 Kam Shueng Rode', '荃湾西站 Tsuen Wan West', '美孚站 Mei Foo', '南昌站 Nam Cheong', '柯士甸站 Austin', '尖东站 East Tsim Sha Tsui', '红磡站 Hung Hom']
['荃湾Tsuen Wan', '大窝口 Tai Wo Hau', '葵兴 Kuai Hing', '葵芳Kuai Fong', '荔景Lai King', '美孚Mei Foo', '荔枝角 Lai Chi Kok', '长沙湾Cheung Sha Wan', '深水埗Sham Shui Po', '太子 Prince Edward', '旺角 Mong Kok', '油麻地Yau Ma Tei', '佐敦Jordan', '尖沙咀Tsim Sha Tsoi', '金钟 Admiralty', '中环 Central']
['调景岭', '油塘', '蓝田', '观塘', '牛头角', '九龙湾', '彩虹', '钻石山', '黄大仙', '乐富', '九龙塘', '石硖尾', '太子', '旺角', '油麻地', '何文田', '黄埔']
['添马', '会展站', '铜锣湾北站', '炮台山站', '北角站', '鲗鱼涌站', '太古站', '西湾河站', '筲箕湾站', '杏花邨站', '柴湾站']
['屯门站', '兆康站', '天水围站', '朗屏站', '元朗站', '锦上路站', '荃湾西站', '美孚站', '南昌站', '柯士甸站', '尖东站'

'马鞍山线'

In [9]:
'''
统一不同线路站点名字，去掉'站'
'''
for line in all_line_stations:
    stations = all_line_stations.get(line)
    all_line_stations[line] = [station[:-1] if '站' in station else station for station in stations]

In [10]:
 all_line_stations

{'东铁线': ['罗湖',
  '落马洲',
  '上水',
  '粉岭',
  '太和',
  '大埔墟',
  '大学',
  '马场',
  '火炭',
  '沙田',
  '大围',
  '九龙塘',
  '旺角东',
  '红磡'],
 '观塘线': ['调景岭',
  '油塘',
  '蓝田',
  '观塘',
  '牛头角',
  '九龙湾',
  '彩虹',
  '钻石山',
  '黄大仙',
  '乐富',
  '九龙塘',
  '石硖尾',
  '太子',
  '旺角',
  '油麻地',
  '何文田',
  '黄埔'],
 '港岛线': ['坚尼地城',
  '香港大学',
  '西营盘',
  '上环',
  '中环',
  '金钟',
  '湾仔',
  '铜锣湾',
  '天后',
  '炮台山',
  '北角',
  '鲗鱼涌',
  '太古',
  '西湾河',
  '筲箕湾',
  '杏花邨',
  '柴湾'],
 '东涌线': ['添马',
  '会展',
  '铜锣湾北',
  '炮台山',
  '北角',
  '鲗鱼涌',
  '太古',
  '西湾河',
  '筲箕湾',
  '杏花邨',
  '柴湾'],
 '西铁线': ['屯门',
  '兆康',
  '天水围',
  '朗屏',
  '元朗',
  '锦上路',
  '荃湾西',
  '美孚',
  '南昌',
  '柯士甸',
  '尖东',
  '红磡'],
 '荃湾线': ['荃湾',
  '大窝口',
  '葵兴',
  '葵芳',
  '荔景',
  '美孚',
  '荔枝角',
  '长沙湾',
  '深水埗',
  '太子',
  '旺角',
  '油麻地',
  '佐敦',
  '尖沙咀',
  '金钟',
  '中环'],
 '南港岛线': ['海怡半岛', '利东', '黄竹坑', '海洋公园', '金钟'],
 '将军澳线': ['北角', '鲗鱼涌', '油塘', '调景岭', '将军澳', '康城', '坑口', '宝琳'],
 '马鞍山线': ['乌溪沙', '马鞍山', '恒安', '大水坑', '石门', '第一城', '沙田围', '车公庙', '大围'],
 '迪士尼线': ['欣澳站往迪士尼', '迪士尼站往欣澳'],
 '机

### 3.建立搜索图

In [11]:
'''
connection
'''
from collections import defaultdict
station_line_connection = defaultdict(list)

#dfs
for name1 in all_line_stations.keys():
    for each1 in all_line_stations.get(name1):
        for name2 in all_line_stations.keys():
            for each2 in all_line_stations.get(name2):
                if (each1==each2)&(name1!=name2):
                    station_line_connection[each1].append(name1)

In [12]:
stations_connection = defaultdict(list)
for name1 in all_line_stations.keys():
    for each1 in all_line_stations.get(name1):
        idx1 = all_line_stations.get(name1).index(each1)
        if idx1==0:
            cnt_1 = all_line_stations.get(name1)[idx1+1]
            stations_connection[each1].append(' '.join([name1,cnt_1]))
            
        elif idx1 == len(all_line_stations.get(name1))-1:
            cnt_0 = all_line_stations.get(name1)[idx1-1]
            stations_connection[each1].append(' '.join([name1,cnt_0]))
        else:
            cnt_0= all_line_stations.get(name1)[idx1-1]
            cnt_1= all_line_stations.get(name1)[idx1+1]
            stations_connection[each1].append(' '.join([name1,cnt_0]))
            stations_connection[each1].append(' '.join([name1,cnt_1]))
            
        for name2 in all_line_stations.keys():
            if name2 == name1:continue
            for each2 in all_line_stations.get(name2):               
                if each1==each2:
                    idx2 = all_line_stations.get(name2).index(each2)
                    if idx2==0:
                        cnt_4 = all_line_stations.get(name2)[idx2+1]
                        stations_connection[each1].append(' '.join([name2,cnt_4]))
                        
                    elif idx2==len(all_line_stations.get(name2))-1:
                        cnt_3= all_line_stations.get(name2)[idx2-1]
                        stations_connection[each1].append(' '.join([name2,cnt_3]))
                    else:
                        cnt_3= all_line_stations.get(name2)[idx2-1]
                        cnt_4= all_line_stations.get(name2)[idx2+1]
                        stations_connection[each1].append(' '.join([name2,cnt_3]))
                        stations_connection[each1].append(' '.join([name2,cnt_4]))
                else:continue

for name,stations in stations_connection.items():
    stations_connection[name]=(list(set(stations)))

In [13]:
 stations_connection

defaultdict(list,
            {'罗湖': ['东铁线 落马洲'],
             '落马洲': ['东铁线 上水', '东铁线 罗湖'],
             '上水': ['东铁线 落马洲', '东铁线 粉岭'],
             '粉岭': ['东铁线 上水', '东铁线 太和'],
             '太和': ['东铁线 大埔墟', '东铁线 粉岭'],
             '大埔墟': ['东铁线 太和', '东铁线 大学'],
             '大学': ['东铁线 大埔墟', '东铁线 马场'],
             '马场': ['东铁线 火炭', '东铁线 大学'],
             '火炭': ['东铁线 沙田', '东铁线 马场'],
             '沙田': ['东铁线 火炭', '东铁线 大围'],
             '大围': ['东铁线 九龙塘', '马鞍山线 车公庙', '东铁线 沙田'],
             '九龙塘': ['观塘线 乐富', '观塘线 石硖尾', '东铁线 大围', '东铁线 旺角东'],
             '旺角东': ['东铁线 九龙塘', '东铁线 红磡'],
             '红磡': ['西铁线 尖东', '东铁线 旺角东'],
             '调景岭': ['观塘线 油塘', '将军澳线 将军澳', '将军澳线 油塘'],
             '油塘': ['观塘线 调景岭', '观塘线 蓝田', '将军澳线 调景岭', '将军澳线 鲗鱼涌'],
             '蓝田': ['观塘线 油塘', '观塘线 观塘'],
             '观塘': ['观塘线 蓝田', '观塘线 牛头角'],
             '牛头角': ['观塘线 观塘', '观塘线 九龙湾'],
             '九龙湾': ['观塘线 牛头角', '观塘线 彩虹'],
             '彩虹': ['观塘线 钻石山', '观塘线 九龙湾'],
             '钻石山': ['观塘线 黄大仙', '观塘线 彩虹'

### 4.breath first search

In [14]:
def search(start,destination,connection):
    visited=set()
    pathes=[[' '.join(['start',start])]]
    while pathes:
        path = pathes.pop()
        frontier = path[-1]
        
        #print(frontier)
        if frontier in visited:
            continue
        for s in connection[frontier.split()[1]]:
            next_s = s.split()[1]
            if s in path:
                continue
            new_path = path + [s]
            pathes.append(new_path)
            if next_s == destination:
                return new_path
        visited.add(frontier)

In [15]:
line1 = search('中环','旺角',stations_connection)
line2 = search('铜锣湾','天水围',stations_connection)

print(' → ▷'.join(line1))
print(' → ▷'.join(line2))

start 中环 → ▷港岛线 金钟 → ▷港岛线 中环 → ▷荃湾线 金钟 → ▷南港岛线 海洋公园 → ▷南港岛线 金钟 → ▷荃湾线 尖沙咀 → ▷荃湾线 佐敦 → ▷荃湾线 油麻地 → ▷荃湾线 旺角
start 铜锣湾 → ▷港岛线 天后 → ▷港岛线 铜锣湾 → ▷港岛线 湾仔 → ▷港岛线 金钟 → ▷港岛线 中环 → ▷荃湾线 金钟 → ▷南港岛线 海洋公园 → ▷南港岛线 金钟 → ▷荃湾线 尖沙咀 → ▷荃湾线 佐敦 → ▷荃湾线 油麻地 → ▷观塘线 何文田 → ▷观塘线 油麻地 → ▷观塘线 旺角 → ▷荃湾线 太子 → ▷荃湾线 深水埗 → ▷荃湾线 长沙湾 → ▷荃湾线 荔枝角 → ▷荃湾线 美孚 → ▷西铁线 荃湾西 → ▷西铁线 锦上路 → ▷西铁线 元朗 → ▷西铁线 朗屏 → ▷西铁线 天水围
